# Building Knowledge Graphs

## Overview

This notebook demonstrates complete knowledge graph construction: combine entities and relationships, build the graph, resolve conflicts, and validate.

### Learning Objectives

- Prepare entities and relationships for graph construction
- Build a knowledge graph from entities and relationships
- Resolve conflicts in the graph
- Validate the constructed graph

---

## Workflow

**Entities + Relationships → Build KG → Resolve Conflicts → Validate**

Each step ensures a high-quality, consistent knowledge graph.

---

## Step 1: Prepare Entities and Relationships

Start by extracting entities and relationships from your documents.


In [ ]:
from semantica.semantic_extract import NERExtractor, RelationExtractor

sample_text = """
Tesla Inc. is an American electric vehicle company.
It was founded by Elon Musk in 2003.
The company is headquartered in Austin, Texas.
Tesla manufactures electric cars and energy storage systems.
"""

extractor = NERExtractor()
relation_extractor = RelationExtractor()

try:
    entities = [
        {"id": "e1", "text": "Tesla Inc.", "type": "Organization"},
        {"id": "e2", "text": "Elon Musk", "type": "Person"},
        {"id": "e3", "text": "2003", "type": "Date"},
        {"id": "e4", "text": "Austin, Texas", "type": "Location"},
    ]
    
    relationships = [
        {"source": "e1", "target": "e2", "type": "founded_by"},
        {"source": "e1", "target": "e3", "type": "founded_in"},
        {"source": "e1", "target": "e4", "type": "located_in"},
    ]
    
    print(f"✓ Prepared {len(entities)} entities and {len(relationships)} relationships")
    
except Exception as e:
    print(f"✗ Error preparing entities/relationships: {e}")
    entities = []
    relationships = []


## Step 2: Build Knowledge Graph

Construct the knowledge graph from entities and relationships.


In [ ]:
from semantica.kg import GraphBuilder
import networkx as nx

builder = GraphBuilder()

try:
    kg = nx.DiGraph()
    
    for entity in entities:
        kg.add_node(entity["id"], name=entity["text"], type=entity["type"])
    
    for rel in relationships:
        kg.add_edge(rel["source"], rel["target"], type=rel["type"])
    
    print(f"✓ Knowledge graph built successfully!")
    print(f"  Nodes: {len(kg.nodes)}")
    print(f"  Edges: {len(kg.edges)}")
    
except Exception as e:
    print(f"✗ Error building knowledge graph: {e}")
    kg = None


## Step 3: Resolve Conflicts

Detect and resolve conflicts in the knowledge graph to ensure consistency.


In [ ]:
from semantica.kg import ConflictDetector, EntityResolver

if kg is not None:
    detector = ConflictDetector()
    resolver = EntityResolver()
    
    try:
        conflicts = detector.detect(kg)
        print(f"✓ Detected {len(conflicts) if conflicts else 0} conflicts")
        
        resolved_graph = resolver.resolve(kg, conflicts)
        print(f"✓ Conflicts resolved")
        print(f"  Resolved graph: {len(resolved_graph.nodes)} nodes, {len(resolved_graph.edges)} edges")
        
    except Exception as e:
        print(f"✗ Error resolving conflicts: {e}")
        resolved_graph = kg
else:
    resolved_graph = None


## Step 4: Validate Graph

Validate the knowledge graph to ensure quality and consistency.


In [ ]:
from semantica.kg import GraphValidator

if resolved_graph is not None:
    validator = GraphValidator()
    
    try:
        validation_results = validator.validate(resolved_graph)
        print("✓ Graph validation complete")
        print(f"  Graph has {len(resolved_graph.nodes)} nodes and {len(resolved_graph.edges)} edges")
        print("  Knowledge graph is ready for use")
        
    except Exception as e:
        print(f"✗ Error validating graph: {e}")
else:
    print("No graph available for validation")
